# ACP Project - Model Evaluation

## Setup

In [2]:
import warnings, pickle, os
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

from IPython.display import display
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(10,10)})

import shap
import optuna

%load_ext autoreload
%autoreload 1

In [3]:
from dataset import SCIData, SCICols
%aimport utils.evaluation, dataset

In [7]:
from sklearn.model_selection import train_test_split

sci = SCIData.load('data/sci.h5')

scii = (
    SCIData(SCIData.quickload("data/sci_processed.h5").sort_values("AdmissionDateTime"))
    .mandate(SCICols.news_data_raw)
    .derive_critical_event(within=1, return_subcols=True)
    .augment_shmi(onehot=False)
    .omit_redundant()
    .raw_news()
    .derive_ae_diagnosis_stems()
    .categorize()
    .onehot_encode_categories()
)

sci_train, sci_test, _, y_test_mortality, _, y_test_criticalcare = train_test_split(
    scii,
    scii.DiedWithinThreshold,
    scii.CriticalCare,
    test_size=0.33,
    random_state=42,
    shuffle=False,
)
sci_train, sci_test = SCIData(sci_train), SCIData(sci_test)

## Population

In [5]:
pd.DataFrame(
    [
        dict(
            Subset=name,
            Records=df.shape[0],
            Patients=df.PatientNumber.unique().shape[0],
            CriticalEvents=df.CriticalEvent.sum(),
            Deaths=df.DiedWithinThreshold.sum(),
            CriticalCare=df.CriticalCare.sum(),
            Readmissions=df.Readmission.sum(),
        )
        for name, df in {'Total': scii, 'Training Set': sci_train, 'Testing Set': sci_test}.items()
    ]
).set_index('Subset').T

AttributeError: 'SCIData' object has no attribute 'PatientNumber'

1. Data size (total, training, testing, total patients)
2. Outcomes (critical, deaths, icu)
3. Sex (female %), Age (median)
4. Comorbid conditions (median #, max 7 per patient)
5. Presence of vital signs, presence of blood test results, 
6. LOS of hospitalisation 

In [ ]:
scii.Female.value_counts()

True     63578
False    57743
Name: Female, dtype: int64

In [ ]:
scii.AgeBand.value_counts()

80 +            36894
65 - 79         31156
50 - 64         23267
30 - 49         22210
17 - 29          7757
16 and Under       37
Name: AgeBand, dtype: int64

In [ ]:
scii.LOSBand.value_counts()

More than 72 Hrs    54207
Less than 12 Hrs    25238
12 - 24 Hrs         16569
48 - 72 Hrs          9834
24 - 36 Hrs          7950
36 - 48 Hrs          7479
Still In               44
Name: LOSBand, dtype: int64

In [ ]:
scii.AdmitWard.value_counts()

EAU     81131
AEC     18964
AAA     10136
AAAC     6132
HH3      2925
HH1M     2033
Name: AdmitWard, dtype: int64

In [ ]:
sci[sci.notna().all(axis=1)].iloc[0]

SpellSerial                                                              222967_146
PatientNumber                                                                298990
SEQ                                                                              15
Female                                                                         True
Age                                                                              61
AgeBand                                                                     50 - 64
Area                                                                           M28 
AdmissionDateTime                                               2020-10-14 23:18:00
DischargeDateTime                                               2020-12-18 20:40:00
TotalLOS                                                                  64.890278
LOSBand                                                            More than 72 Hrs
Over7Days                                                                   

In [ ]:
scii[SCICols.wards].stack().value_counts()

EAU     114643
AEC      32618
DISL     20437
AAA      15498
AAAC     11355
L6        8346
M2        7772
HH3       7178
HH2       6290
L3        6101
L2        5985
L4        5877
PNDS      5040
L5        4514
HH1M      4112
HCU       3772
MA3       2794
HH4       2406
L8        2233
L7        1722
HB2       1423
HB1       1418
ANU       1411
CCU       1180
HH5       1151
RU        1129
B6        1056
MIU        982
HG         936
B3         911
LIME       877
ASU        836
B8         737
WPW        735
HH7        625
HAEM       567
ETW        566
M2SS       544
STU        445
B7         403
B5         342
HASU       327
CPIU       320
THE3       247
SBU        233
SRU        230
M2S        210
RCV3       200
THE1       169
NSSU       123
C1         105
C3          81
HH6M        76
M2SU        72
HH8         60
B4          58
TAU         54
DCU         51
HAED        50
L1          47
ML          40
SAL         37
HB2M        33
RTU         23
C2          22
RUSS        21
MET       

In [ ]:
sci[(sci[SCICols.diagnoses] == 'Z33.X').any(axis=1)]

,SpellSerial,PatientNumber,SEQ,Female,Age,AgeBand,Area,AdmissionDateTime,DischargeDateTime,TotalLOS,LOSBand,Over7Days,Over14Days,AdmittedAfterAEC,AdmissionMethodDescription,AssessmentAreaAdmission,AssessmentAreaDischarge,AdmissionSpecialty,LastSpecialty,AdmitWard,NextWard2,NextWard3,NextWard4,NextWard5,NextWard6,NextWard7,NextWard8,NextWard9,DischargeWard,AdmitWardLOS,NextWard2LOS,NextWard3LOS,NextWard4LOS,NextWard5LOS,NextWard6LOS,NextWard7LOS,NextWard8LOS,NextWard9LOS,DischargeWardLOS,DiedDuringStay,DiedWithin30Days,AandEPresentingComplaint,AandEMainDiagnosis,AandEArrivalTime,AandEDepartureTime,AandELocation,AandEPatientGroupDescription,MainICD10,SecDiag1,SecDiag2,SecDiag3,SecDiag4,SecDiag5,SecDiag6,MainOPCS4,SecOper1,SecOper2,SecOper3,SecOper4,SecOper5,SecOper6,SpellHRG,HRGDesc,PrimarySpecialtyLocalCode,CareHome,PCT,GPPractice,Haemoglobin,Urea_serum,Sodium_serum,Potassium_serum,Creatinine,PatientTemperatureVenous,pCO2(POC)Venous,pCO2(Tempcorrected)(POC)Venous,PH(POC)Venous,PH(Tempcorrected)(POC)Venous,pO2(POC)Venous,pO2(Tempcorrected)(POC)Venous,NEWS_score,NewsCreatedWhen,NewsTouchedWhen,NewsAuthoredDtm,Respiration_rate,NEWS_resp_rate_score,AssistedBreathing,BreathingDevice,NEWS_device_air_score,O2_saturation,Oxygen_flow_rate,NEWS_O2_sat_score,Temperature,NEWS_temperature_score,LyingDown,SystolicBP,DiastolicBP,NEWS_BP_score,HeartRate,NEWS_heart_rate_score,AVCPU_Alert,NEWS_level_of_con_score,Pain,Nausea,VomitingSinceLastRound
1772,1839457_136,2161286,24,True,22,17 - 29,M38,2019-03-26 16:30:00,2019-03-26 23:00:00,0.270833,Less than 12 Hrs,False,False,False,EMERGENCY A+E,False,False,ACCIDENT & EMERGENCY,ACCIDENT & EMERGENCY,AEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEC,0.270833,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.270833,False,False,Headache,migraine (disorder),2019-03-26 12:36:00,2019-03-26 16:30:00,Majors 2,MEDICAL,R51.X,M54.22,I82.0,F60.3,Z33.X,Z86.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AA31E,"Headache, Migraine or Cerebrospinal Fluid Leak...",AE,False,01G,P87630,112.0,NaN,136.0,4.4,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2019-03-26 16:58:33,2019-03-26 16:58:33,2019-03-26 16:54:00,16.0,0.0,False,A - Air,0.0,98.0,0.0,0.0,36.9,0.0,False,120.0,80.0,0.0,98.0,1.0,True,0.0,0.0,False,False
1936,1839457_126,2161286,22,True,22,17 - 29,M38,2019-01-18 03:36:00,2019-01-18 10:14:00,0.276389,Less than 12 Hrs,False,False,False,EMERGENCY A+E,True,True,ACCIDENT & EMERGENCY,ACCIDENT & EMERGENCY,EAU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAU,0.276389,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.276389,False,False,Back Pain,injury of muscle of lower back (disorder),2019-01-17 23:36:00,2019-01-18 03:36:00,Majors 2,OTHER THAN ABOVE,M54.5,R32.X,F17.1,Z33.X,NaN,NaN,NaN,U21.1,Y98.2,Z66.5,Z66.8,NaN,NaN,NaN,HC32K,"Low Back Pain without Interventions, with CC S...",AE,False,01G,P87630,119.0,2.4,137.0,4.6,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2019-01-18 03:39:52,2019-01-18 03:39:52,2019-01-18 03:39:00,17.0,0.0,False,A - Air,0.0,100.0,0.0,0.0,37.4,0.0,True,114.0,68.0,0.0,98.0,1.0,True,0.0,0.0,False,False
7971,1296047_86,609520,10,True,23,17 - 29,M30,2015-12-01 19:21:00,2015-12-04 12:12:00,2.702083,48 - 72 Hrs,False,False,False,EMERGENCY A+E,False,False,ACCIDENT & EMERGENCY,GENERAL MEDICINE,AAAC,EAU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L6,0.013889,0.141667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.546528,False,False,Shortness of breath in adults,Shortness of Breath in Adults,2015-12-01 16:14:00,2015-12-01 19:21:00,Majors,OTHER THAN ABOVE,J45.9,K29.7,F17.1,Z33.X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA12Z,Asthma or Wheezing,AE,False,01G,P87020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2015-12-03 06:35:05,2015-12-03 06:35:05,2015-12-03 06:35:02,18.0,0.0,False,A - Air,0.0,98.0,NaN,0.0,36.9,0.0,True,96.0,44.0,2.0,111.0,2.0,True,0.0,0.0,False,False
8378,1608472_135,1944093,10,True,31,30 - 49,M28,2021-08-13 14:09:00,2021-08-13 16:45:00,0.108333,Less than 12 Hrs,False,False,True,BOOKED ADMISSION,False,False,ACUTE INTERNAL MEDICINE,ACUTE INTERNAL MEDICINE,AEC,NaN,NaN,NaN,NaN,NaN,Na

In [8]:
diagnosis = sci.AandEMainDiagnosis
complaint = sci.AandEPresentingComplaint
stems = ['confus', 'weak', 'found', 'fof', 'dementia', 'discharged', 'sob', 'unwitnessed', 'gcs', 'diarrh', 'vomit', 'collaps', 'sudden', 'woke', 'dizz', 'tight', 'head', 'fall', 'fell', 'pain', 'bang', 'mobility', 'cope', 'coping', 'weak', 'deterio']

In [ ]:
pd.get_dummies(diagnosis.str.lower().str.extract('('+'|'.join(stems)+')', expand=False)).groupby(level=0).sum().sum(axis=0)

collaps      1571
confus        402
cope          891
dementia      261
diarrh        369
dizz          126
fall         1829
head         3943
mobility      113
pain        10308
sudden          2
tight           7
vomit         359
dtype: int64

In [21]:
sci.clean_ae_text().derive_ae_diagnosis_stems(onehot=True)

,SpellSerial,PatientNumber,SEQ,Female,Age,AgeBand,Area,AdmissionDateTime,DischargeDateTime,TotalLOS,LOSBand,Over7Days,Over14Days,AdmittedAfterAEC,AdmissionMethodDescription,AssessmentAreaAdmission,AssessmentAreaDischarge,AdmissionSpecialty,LastSpecialty,AdmitWard,NextWard2,NextWard3,NextWard4,NextWard5,NextWard6,NextWard7,NextWard8,NextWard9,DischargeWard,AdmitWardLOS,NextWard2LOS,NextWard3LOS,NextWard4LOS,NextWard5LOS,NextWard6LOS,NextWard7LOS,NextWard8LOS,NextWard9LOS,DischargeWardLOS,DiedDuringStay,DiedWithin30Days,AandEArrivalTime,AandEDepartureTime,AandEPatientGroupDescription,MainICD10,SecDiag1,SecDiag2,SecDiag3,SecDiag4,SecDiag5,SecDiag6,MainOPCS4,SecOper1,SecOper2,SecOper3,SecOper4,SecOper5,SecOper6,SpellHRG,HRGDesc,PrimarySpecialtyLocalCode,CareHome,PCT,GPPractice,Haemoglobin,Urea_serum,Sodium_serum,Potassium_serum,Creatinine,PatientTemperatureVenous,pCO2(POC)Venous,pCO2(Tempcorrected)(POC)Venous,PH(POC)Venous,PH(Tempcorrected)(POC)Venous,pO2(POC)Venous,pO2(Tempcorrected)(POC)Venous,NEWS_score,NewsCreatedWhen,NewsTouchedWhen,NewsAuthoredDtm,Respiration_rate,NEWS_resp_rate_score,AssistedBreathing,BreathingDevice,NEWS_device_air_score,O2_saturation,Oxygen_flow_rate,NEWS_O2_sat_score,Temperature,NEWS_temperature_score,LyingDown,SystolicBP,DiastolicBP,NEWS_BP_score,HeartRate,NEWS_heart_rate_score,AVCPU_Alert,NEWS_level_of_con_score,Pain,Nausea,VomitingSinceLastRound,AandEPresentingComplaint__abdominal pain in adults,AandEPresentingComplaint__abnormal bloods,AandEPresentingComplaint__abscesses and local infections,AandEPresentingComplaint__allergy,AandEPresentingComplaint__apparently drunk,AandEPresentingComplaint__assault,AandEPresentingComplaint__asthma,AandEPresentingComplaint__back pain,AandEPresentingComplaint__behaving strangely,AandEPresentingComplaint__bites and stings,AandEPresentingComplaint__burns and scalds,AandEPresentingComplaint__chest infection,AandEPresentingComplaint__chest pain,AandEPresentingComplaint__collapsed adult,AandEPresentingComplaint__covid,AandEPresentingComplaint__dental problems,AandEPresentingComplaint__diabetes,AandEPresentingComplaint__diarrhoea and vomiting,AandEPresentingComplaint__ear problems,AandEPresentingComplaint__eye problems,AandEPresentingComplaint__facial problems,AandEPresentingComplaint__falls,AandEPresentingComplaint__fits,AandEPresentingComplaint__foreign body,AandEPresentingComplaint__gi bleeding,AandEPresentingComplaint__head injury,AandEPresentingComplaint__headache,AandEPresentingComplaint__limb problems,AandEPresentingComplaint__major trauma,AandEPresentingComplaint__mental illness,AandEPresentingComplaint__neck pain,AandEPresentingComplaint__other,AandEPresentingComplaint__overdose and poisoning,AandEPresentingComplaint__palpitations,AandEPresentingComplaint__pneumonia,AandEPresentingComplaint__pregnancy,AandEPresentingComplaint__pv bleeding,AandEPresentingComplaint__rashes,AandEPresentingComplaint__red flag,AandEPresentingComplaint__reduced gcs,AandEPresentingComplaint__self-harm,AandEPresentingComplaint__sepsis,AandEPresentingComplaint__shortness of breath in adults,AandEPresentingComplaint__shortness of breath in children,AandEPresentingComplaint__sore throat,AandEPresentingComplaint__testicular pain,AandEPresentingComplaint__torso injury,AandEPresentingComplaint__urinary problems,AandEPresentingComplaint__uti,AandEPresentingComplaint__wounds,AandEMainDiagnosis__collaps,AandEMainDiagnosis__confus,AandEMainDiagnosis__cope,AandEMainDiagnosis__dementia,AandEMainDiagnosis__diarrh,AandEMainDiagnosis__dizz,AandEMainDiagnosis__fall,AandEMainDiagnosis__head,AandEMainDiagnosis__mobility,AandEMainDiagnosis__pain,AandEMainDiagnosis__sudden,AandEMainDiagnosis__tight,AandEMainDiagnosis__vomit
0,280233_1206,441440,143,True,53,50 - 64,M30,2020-07-05 21:29:00,2020-07-06 10:01:00,0.522222,12 - 24 Hrs,False,False,False,EMERGENCY A+E,True,True,ACUTE INTERNAL MEDICINE,ACUTE INTERNAL MEDICINE,EAU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAU,0.522222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.522222,Fal